## Cleaning file for 2019-2020

In [1]:
import pandas as pd
import numpy as np

In [2]:
roster_data_1920 = pd.read_stata('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/00_source_data/1920_roster.dta')
roster_data_1920.head()

,hhcode,psu,province,region,district,idc,sb1q2,sb1q3,sb1q4,sb1q5,age,sb1q62,sb1q63,sb1q64,sb1q7,sb1q8,sb1q9,sb1q10,sb1q11
0,101100101,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,58,0,0,1961.0,currently married,2.0,98.0,99.0,yes
1,101100102,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,64,0,0,1955.0,currently married,2.0,98.0,10.0,yes
2,101100103,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,35,0,0,1984.0,currently married,2.0,98.0,99.0,yes
3,101100104,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,56,0,0,1963.0,currently married,2.0,98.0,99.0,yes
4,101100105,1011001,khyber pakhtunkhwa,rural,abbottabad,1,head,NaN,male,present,50,0,0,1969.0,currently married,2.0,98.0,98.0,yes


In [3]:
# renaming relevant variables from roster
roster_data_1920['sex'] = roster_data_1920['sb1q4']
roster_data_1920['marital_status'] = roster_data_1920['sb1q7']
roster_data_1920.sample(4)

,hhcode,psu,province,region,district,idc,sb1q2,sb1q3,sb1q4,sb1q5,...,sb1q62,sb1q63,sb1q64,sb1q7,sb1q8,sb1q9,sb1q10,sb1q11,sex,marital_status
654227,232200621,2322006,punjab,urban,sahiwal,5,son/daughter,family elder,male,present,...,0,0,2002.0,unmarried / never married,NaN,1.0,2.0,yes,male,unmarried / never married
626839,211200107,2112001,punjab,urban,hafizabad,5,grand child,main economic provider,female,present,...,5,1,2014.0,unmarried / never married,NaN,3.0,4.0,yes,female,unmarried / never married
217330,214105605,2141056,punjab,rural,jhang,2,son/daughter,is oldest male in the house,male,present,...,0,0,1974.0,currently married,3.0,1.0,99.0,yes,male,currently married
795319,313201418,3132014,sindh,urban,khairpur,7,son/daughter in law,main economic provider,female,present,...,0,0,1995.0,currently married,6.0,99.0,99.0,yes,female,currently married


In [5]:
B_relevant = roster_data_1920[['hhcode', 'idc','age', 'sex', 'marital_status']].copy()
B_relevant.sample(5)

,hhcode,idc,age,sex,marital_status
631759,216101121,5,9,male,unmarried / never married
672487,317100922,5,10,female,unmarried / never married
809714,125200521,8,3,female,unmarried / never married
99976,233200719,1,47,male,currently married
376414,216105713,3,5,male,unmarried / never married


In [9]:
educ_data = pd.read_stata('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/00_source_data/1920_secc1.dta',
convert_categoricals=False)
educ_data.columns

Index(['hhcode', 'psu', 'province', 'region', 'idc', 'district', 'sc1q1a',
       'sc1q2a', 'sc1q3a', 'sc1q4a', 'sc1q01', 'sc1q02', 'sc1q03', 'sc1q04',
       'sc1q05', 'sc1q06', 'sc1q07', 'sc1q08', 'sc1q09', 'sc1q10', 'sc1q11',
       'sc1q12', 'sc1q13', 'sc1q14', 'sc1q15', 'sc1q16', 'sc1q17', 'sc1q18',
       'sc1q19a', 'sc1q19b', 'sc1q19c', 'sc1q19d', 'sc1q19e', 'sc1q19f',
       'sc1q19g', 'sc1q19h', 'sc1q19i'],
      dtype='object')

In [12]:
educ_data.sc1q01.value_counts()

1    355074
2    244505
3    214697
Name: sc1q01, dtype: int64

In [13]:
# Create variable for ever_admitted and currently attending
educ_data['ever_admitted'] = np.where(educ_data.sc1q01 == 1, 'no', 'yes')
educ_data.ever_admitted.value_counts()
educ_data['currently_enrolled'] = np.where(educ_data.sc1q01 == 3, 'yes', 'no')
educ_data.currently_enrolled.value_counts()

no     599579
yes    214697
Name: currently_enrolled, dtype: int64

In [14]:
C_relevant = educ_data[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province"
    ]
].copy()

C_relevant.sample(5)

,hhcode,idc,ever_admitted,currently_enrolled,region,province
37405,108100928,5,no,no,1,1
535988,234105028,1,no,no,1,2
448137,227100418,1,yes,no,1,2
195119,203101324,4,yes,yes,1,2
672915,317100929,2,no,no,1,3


In [15]:
# MERGE 
years19_20 = pd.merge(B_relevant, C_relevant, on=["hhcode", 'idc'], how='inner', indicator=False)
# Evaluate merge by checking value counts for currently enrolled individuals
years19_20.currently_enrolled.value_counts()

no     599579
yes    214697
Name: currently_enrolled, dtype: int64

In [17]:
# Select people in school enrollment age (I would say 4-10 years old)
years19_20 = years19_20.loc[(years19_20["age"] >= 4)&(years19_20["age"] <= 15)]


In [20]:
years19_20.province.value_counts()

2    128456
1     65030
3     57372
4     30042
Name: province, dtype: int64

In [21]:
# Re-spell the names
years19_20['province'].replace(2, 'Punjab', inplace = True)
years19_20['province'].replace(1, 'KPK', inplace = True)
years19_20['province'].replace(3, 'Sindh', inplace = True)
years19_20['province'].replace(4, 'Balochistan', inplace = True)
years19_20.province.value_counts()

Punjab         128456
KPK             65030
Sindh           57372
Balochistan     30042
Name: province, dtype: int64

In [22]:
# Lets see which hh codes we have:
years19_20["hhcode"] = years19_20["hhcode"].astype("str")
years19_20['subprovince code']= years19_20['hhcode'].apply(lambda x: x[0:4])
years19_20["subprovince code"] = years19_20["subprovince code"].astype("int")
years19_20.head()

,hhcode,idc,age,sex,marital_status,ever_admitted,currently_enrolled,region,province,subprovince code
7204,109100622,1,15,male,unmarried / never married,yes,yes,1,KPK,1091
12381,115100530,1,15,male,unmarried / never married,yes,yes,1,KPK,1151
17584,120104015,1,15,female,unmarried / never married,yes,no,1,KPK,1201
29290,201102822,1,15,male,unmarried / never married,yes,yes,1,Punjab,2011
30235,202100124,1,10,female,unmarried / never married,yes,yes,1,Punjab,2021


In [25]:
# Add stratum data
stratum_1920 = pd.read_csv('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/00_source_data/stratum1920.csv')
stratum_1920.sample(2)

,subprovince,subprovince code
225,Jamshoro,3061
265,Loralai,4171


In [26]:
# Merge stratum data with edu and sex data
years19_20_ = pd.merge(years19_20, stratum_1920, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years19_20_._merge.value_counts()

both          280900
left_only          0
right_only         0
Name: _merge, dtype: int64

In [27]:
years19_20_.drop(columns= ['_merge'], inplace = True)
years19_20_.head()

,hhcode,idc,age,sex,marital_status,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince
0,109100622,1,15,male,unmarried / never married,yes,yes,1,KPK,1091,Hangu
1,109100106,2,14,male,unmarried / never married,yes,yes,1,KPK,1091,Hangu
2,109100118,2,6,female,unmarried / never married,yes,yes,1,KPK,1091,Hangu
3,109100122,2,15,male,unmarried / never married,yes,yes,1,KPK,1091,Hangu
4,109100204,2,10,male,unmarried / never married,yes,yes,1,KPK,1091,Hangu


In [28]:
# Indicator for year
years19_20_['Year'] = 2019


In [29]:
years19_20_.marital_status.value_counts()

unmarried / never married                         280713
nikkah solemnized but rukhsati not taken place        99
currently married                                     83
separated                                              3
divorced                                               2
0                                                      0
widow / widower                                        0
Name: marital_status, dtype: int64

In [30]:
# Recode marital status
years19_20_.marital_status.replace('unmarried / never married', 1, inplace = True)
years19_20_.marital_status.replace('currently married', 2, inplace = True)
years19_20_.marital_status.replace('widow / widower', 3, inplace = True)
years19_20_.marital_status.replace('separated', 4, inplace = True)
years19_20_.marital_status.replace('divorced', 4, inplace = True)
years19_20_.marital_status.replace('nikkah solemnized but rukhsati not taken place', 5, inplace = True)
years19_20_.marital_status.value_counts()

1    280713
5        99
2        83
4         5
0         0
3         0
Name: marital_status, dtype: int64

In [34]:
years19_20_.marital_status = years19_20_.marital_status.astype('int')
years19_20_.region.value_counts()

1    207443
2     73457
Name: region, dtype: int64

In [35]:
#Re-code region
years19_20_.region.replace(1, 'rural', inplace = True)
years19_20_.region.replace(2, 'urban', inplace = True)


In [36]:
years19_20_.dtypes

hhcode                  object
idc                       int8
age                      int16
sex                   category
marital_status           int64
ever_admitted           object
currently_enrolled      object
region                  object
province                object
subprovince code         int64
subprovince             object
Year                     int64
dtype: object

In [ ]:
# Save to csv
years19_20_.to_csv('/Users/preetkhowaja/Documents/midssp2022/unifying/uds-2022-ids-701-team-3/10_data_cleaning/clean_preet/years19_20_.csv', index_label='id')